In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch

# Импорт зависимостей из проекта
from src.neural.models.kenga import KengaConfig, KengaModel
from src.neural.training import KengaTrainer
from src.neural.data import TextDataset
from src.utils.tokenizer import Tokenizer

# Создаем необходимые директории
os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)

# Создаем файлы с обучающими данными, если их нет
if not os.path.exists("data/train.txt"):
    with open("data/train.txt", "w", encoding="utf-8") as f:
        f.write("This is a sample sentence.\nThis is another sentence.\n")

if not os.path.exists("data/val.txt"):
    with open("data/val.txt", "w", encoding="utf-8") as f:
        f.write("This is a sample sentence.\nThis is another sentence.\n")

# Инициализируем токенизатор и обучаем его
tokenizer = Tokenizer()
train_files = ["data/train.txt"]
val_files = ["data/val.txt"]
tokenizer.train(train_files)
tokenizer.save("models", "kenga")

def generate_tokens(files):
    tokens = []
    for file in files:
        with open(file, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    t = tokenizer.encode(line)
                    if t:
                        tokens.append(t)
    return tokens

train_tokens = generate_tokens(train_files)
val_tokens = generate_tokens(val_files)

train_dataset = TextDataset(train_tokens)
val_dataset = TextDataset(val_tokens)

# Конфигурация модели и создание модели
config = KengaConfig()
config.vocab_size = tokenizer.vocab_size
model = KengaModel(config)

train_config = {
    "lr": 1e-4,
    "batch_size": 4,
    "epochs": 5,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "model_path": "models/kenga.pth"
}

# Создаем объект тренера и запускаем обучение
trainer = KengaTrainer(model, train_dataset, val_dataset, train_config)
trainer.train()


In [ ]:
# Если вы настроили git-креденциалы в Colab, можно автоматически отправить чекпоинт в репозиторий
!git config --global user.email "your_email@example.com"
!git config --global user.name "Your Name"
!git add models/kenga.pth
!git commit -m "Обновленный чекпоинт после обучения в Colab"
!git push origin master
